In [1]:
from sglang import function, system, user, assistant, gen, set_default_backend, RuntimeEndpoint

from utils.utils import(
    read_jsonl,
    get_majority_answer,
    extract_rawdata_answer,
    extract_completion_answer,
    few_shot_prompt, INVALID_ANS,
)

In [2]:
gsm_8k_test = read_jsonl("./data/test.jsonl")

In [ ]:
@function
def multi_turn_question(s, prompt, temperature = 1, **kwargs):
    s += prompt
    s += gen("A", max_tokens=200, temperature=temperature,stop="\nQ:")
    
@function
def multiple_sampling(s,prompt, sampling_num, temperature = 1, **kwargs):
# (s, id, question, sampling_num, max_tokens, ground_truth_answer, temperature, model_type):
    s += prompt
    forks = s.fork(sampling_num)
    forks += gen("A", max_tokens=512, temperature=temperature,stop="\nQ:")
    answers = []
    for state in forks:
        answers += [state["A"]]
    return answers

set_default_backend(RuntimeEndpoint("http://localhost:30000"))
sampling_num = 10
input_data_list = [
    {
        "prompt" : f"{few_shot_prompt}\n\n\nQ: {gsm_8k_item['question']}\nA:", 
        "sampling_num" : sampling_num, 
     } for gsm_8k_item in gsm_8k_test
]
gt_ans_list = [float(extract_rawdata_answer(gsm_8k_item['answer'])) for gsm_8k_item in gsm_8k_test]
num_threads = 10
# states = multiple_sampling.run_batch(
#     input_data_list[:10],
#     num_threads=num_threads, progress_bar=True
# )
states = multi_turn_question.run_batch(
    input_data_list[:10],
    num_threads=num_threads, progress_bar=True
)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


In [10]:
import json

with open(f"./result/model_output.json", "w+") as fp:
    json.dump(
        [state.ret_value for state in states],
        fp
    )


In [ ]:
pre_ans_list = [get_majority_answer([extract_completion_answer(i) for i in state.ret_value ]) for state in states]

In [7]:
correct_count = 0
all_testcase_count = 0
for pre_ans, gt_ans in zip(pre_ans_list, gt_ans_list):
    if abs(pre_ans - gt_ans) < 1e-3:
        correct_count += 1
    all_testcase_count += 1

correct_count / all_testcase_count

0.7

In [22]:
states[0]

ProgramState(Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.


Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.


Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.


Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So h